In [ ]:
import re
import pandas as pd
from datetime import datetime

def is_insert(line):
    return 'INSERT INTO' in line or False

def is_create(line):
    return 'CREATE TABLE' in line or False

def is_finish(line):
    return ';' in line or False

def is_primary(line):
    return 'PRIMARY KEY' in line or False

def get_values(line):
    return line.partition(' VALUES ')[2]

def get_table_name_insert(line):
    match = re.search('INSERT INTO `([0-9_a-zA-Z]+)`', line)
    if match:
        return match.group(1)
    else:
        print(line)

def get_name_categorie_type(line):
    match = re.search('int\([0-9]+\)', line)
    if match:
        return "int"
    else:
        match = re.search('char', line)
        if match:
            return match.group(0)
        else:
            match = re.search('date', line)
            if match:
                return match.group(0)
            else:
                match = re.search('time', line)
                if match:
                    return match.group(0)
                else:
                    match = re.search('text', line)
                    if match:
                        return match.group(0)
                    else:
                        print(line)
              
def get_table_name_create(line):
    match = re.search('CREATE TABLE IF NOT EXISTS `([0-9_a-zA-Z]+)`', line)
    if match:
        return match.group(1)
    else:
        print(line)
        
def get_name_categorie(line):
    match = re.search('`([0-9_a-zA-Z]+)`', line)
    if match:
        return match.group(1)
    else:
        print(line)
      
def get_columns(line):
    match = re.search('INSERT INTO `.*` \(([^\)]+)\)', line)
    if match:
        return list(map(lambda x: x.replace('`', '').strip(), match.group(1).split(',')))

def parse_line(line):
    if line[0] == "(": 
        line = line[1:-2]
        line = line.split(", ")
    return line
    
def sqltopanda(filename):
    d = {}
    type_categorie = {}
    values = []
    table_name = []
    columns = []
    insert = False
    create = False
    reader = open(filename, 'r', encoding='utf-8')
    contenu = reader.read()
    lines = contenu.split("\n")
    j =0
    
    for line in lines:
        if is_insert(line):
            insert = True
        elif is_create(line):
            table = get_table_name_create(line)
            d[get_table_name_create(line)] = pd.DataFrame()
            type_categorie[get_table_name_create(line)] = []
            create = True
        elif is_finish(line) and create:
            create = False
        if create and not is_create(line) and not is_primary(line):
            d[table][get_name_categorie(line)] = ""
            type_categorie[table].append(get_name_categorie_type(line))
        if (insert and not is_insert(line)) or (is_finish(line) and insert):
            dictio = {}
            values = parse_line(line)         
            for i in range(len(values)):
                if values[i] == "NULL":
                    d[table][list(d[table].columns)[i]] = values[i]
                elif type_categorie[table][i] == "int":
                    values[i] = int(values[i])
                elif type_categorie[table][i] == "char":
                    values[i] = str(values[i])
                elif type_categorie[table][i] == "date":
                    values[i] = datetime.strptime(values[i], "'%Y-%m-%d'").date()
                elif type_categorie[table][i] == "time":
                    values[i] = datetime.strptime(values[i], "'%H:%M:%S'").time()
                else:
                    print(type_categorie[table][i], values[i], type(values[i]))
                dictio[d[table].columns[i]] = values[i]
            d[table] = d[table].append(dictio, ignore_index=True)
            if is_finish(line) and insert:
                insert = False
    return d, type_categorie

# current date and time
deb = datetime.now()
sql, type_categorie = sqltopanda("traceforum.sql")
fin = datetime.now()
print("déubt =", sql)
print("fin =", type_categorie)

In [5]:
#---------------------------------P A R T I E      D Y L A N       P A S     F I N I -----------------------------------------


def nbActions(Action, Utilisateur):
    nbMessage = sql['transition'][(sql['transition']['Titre'] == Action) & (sql['transition']['Utilisateur'] == Utilisateur)].count()['IDTran']


In [6]:
Action = ["'Répondre à un message'", "'Poster un nouveau message'"]


for utilisateur in sql['transition']["Utilisateur"].unique().tolist():
    for act in Action:
        nbActions(act, utilisateur)


In [7]:
def postsUtilisateur(utilisateur):
    tab = []
    attribut = sql['transition']['Attribut'][(sql['transition']['Titre'] == "'Poster un nouveau message'") & (sql['transition']['Utilisateur'] == utilisateur)]
    for attr in attribut:
        tab.append(str(attr).split(",")[1].split("\n")[0].split("=")[1][:-1])
    return(tab)
    
    
def getReponsesPost(idParent):
    utilisateur = sql['transition']['Utilisateur'][(sql['transition']['Attribut'].str.contains('IDParent='+idParent)) & (sql['transition']['Titre'] == "'Répondre à un message'")]
    attribut = sql['transition']['Attribut'][(sql['transition']['Attribut'].str.contains('IDParent='+idParent))]
    if(attribut.empty):
        return ("", "")
    idMessage = str(attribut).split(",")[1].split("\n")[0].split("=")[1]
    return (utilisateur, idMessage)


In [10]:
#pour chaque utilisateur
import numpy as np
deb = datetime.now()
utilisateurs = sql['transition']["Utilisateur"].unique().tolist()
for l in range(len(utilisateurs)):
    print("\n Pour l'utilisateur :" + utilisateurs[l] +"\n" )
    #on initialise la matrice de résultat
    a = np.zeros(shape=(1,len(utilisateurs)))
    tableauResultat = pd.DataFrame(a, index = ['l1'], columns = utilisateurs)
    #on recupere les posts qu'il a créé
    posts = postsUtilisateur(utilisateurs[l])
    
    #pour chaque post qu'il a créé
    for i in range(len(posts)):
        tableauUsers = []
        user, idMessage = getReponsesPost(posts[i])
        #tant qu'il y a des messages on déroule le fil de la discussion
        while idMessage != "" :  
            #on récupere l'identifiant du message et l'utilisateur qui a posté le message
            tableauUsers.append(user.values[0])
            user, idMessage = getReponsesPost(idMessage)
        tableauUsers = list(set(tableauUsers))
       
        # on ajoute 1 dans la matrice si l'utilisateur a répondu au post
        for j in range(len(utilisateurs)):
            for k in range(len(tableauUsers)):
                if tableauUsers[k] == utilisateurs[j]:
                    tableauResultat.at['l1', utilisateurs[j]] = (1 + tableauResultat.at['l1', utilisateurs[j]])
   
    # on calcule la matrice en pourcentage
    if (len(posts) != 0) :
        tableauResultat = (tableauResultat * 100) / len(posts)
    print(tableauResultat)
fin = datetime.now()
print(deb)
print(fin)


 Pour l'utilisateur :'tdelille'

    'tdelille'    'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1   52.941176  5.882353        48.529412  29.411765          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0        0.0         0.0        23.529412  ...         0.0   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere'  \
l1       0.0       0.0      0.0        0.0        0.0               0.0   

    'shernu'  'xjmechain'  'ntricoit'  
l1       0.0          0.0         0.0  

[1 rows x 23 columns]

 Pour l'utilisateur :'mmay'

    'tdelille'     'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1   28.571429  42.857143        14.285714  28.571429          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0        0.0         0.0        14.285714  ...         0.0   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere' 

    'tdelille'  'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1       3.125    6.25              0.0        0.0          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0      9.375      34.375              0.0  ...      21.875   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere'  \
l1    28.125       0.0      0.0        0.0        0.0               0.0   

    'shernu'  'xjmechain'  'ntricoit'  
l1       0.0          0.0         0.0  

[1 rows x 23 columns]

 Pour l'utilisateur :'psalam'

    'tdelille'  'mmay'  'ddaniaupotter'  'gachort'  'sebastien'  'admin'  \
l1         0.0     0.0              0.0        0.0          0.0      0.0   

    'madeth'  'cdejean'  'fdarriet'  'mwollenburger'  ...  'jvasseur'  \
l1       0.0        0.0         0.0              0.0  ...         0.0   

    'mdanet'  'psalam'  'guest'  'cevanen'  'cfroger'  'adebeuckelaere'  \
l1       0.0      50.0      0.0      10

In [3]:
def getReponses(action, utilisateur):
     return(sql['transition'][(sql['transition']['Titre'] == action) & (sql['transition']['Utilisateur'] == utilisateur)])

def getMessage(idMsg):
     return(sql['transition'][(sql['transition']['Titre'] == 'Répondre à un message') |
                              (sql['transition']['Titre'] == 'Poster un nouveau message') &
                              (sql['transition']['Attribut'].str.contains(idMsg))])

from datetime import datetime, date

def getDifferenceMessages(dateMessage, timeMessage, dateParent, timeParent):
    print(datetime.combine(date.today(), timeParent) - datetime.combine(date.today(), timeMessage))

In [4]:
#for Utilisateur in sql['transition']["Utilisateur"].unique().tolist():
reponses = getReponses("'Répondre à un message'", "'tdelille'")
print(reponses)

for reponse in reponses.values:
    dateMessage = reponse[4]
    timeMessage = reponse[5]
    timeParent = reponse[2][5]
    idParent = reponse[3].split(",")[1].split("=")[1]
    print(idParent)
    msgParent = getMessage(idParent)
    dateParent = msgParent.values[2][4]
    timeParent = msgParent.values[2][5]
    getDifferenceMessages(dateMessage, timeMessage, dateParent, timeParent)

NameError: name 'sql' is not defined

In [239]:
getMessage("1231")


,IDTran,Utilisateur,Titre,Attribut,Date,Heure,Delai,RefTran,Commentaire
689,783,'ddaniaupotter','Afficher le contenu d''un message',"'IDForum=7,IDMsg=1231'",2009-02-16,12:02:50,NULL,0,NULL
712,812,'tdelille','Afficher le contenu d''un message',"'IDForum=7,IDMsg=1231'",2009-02-16,12:28:55,NULL,0,NULL
721,825,'tdelille','Répondre à un message',"'IDForum=7,IDMsg=1239,IDParent=1231'",2009-02-16,12:43:37,NULL,0,NULL
722,826,'tdelille','Citer un message',"'IDForum=7,IDMsg=1231'",2009-02-16,12:43:37,NULL,825,NULL
731,839,'tdelille','Afficher le contenu d''un message',"'IDForum=7,IDMsg=1231'",2009-02-16,12:58:16,NULL,0,NULL
734,842,'tdelille','Répondre à un message',"'IDForum=7,IDMsg=1242,IDParent=1231'",2009-02-16,13:01:04,NULL,0,NULL
735,843,'tdelille','Citer un message',"'IDForum=7,IDMsg=1231'",2009-02-16,13:01:04,NULL,842,NULL
2023,2202,'ddaniaupotter','Afficher le contenu d''un message',"'IDForum=7,IDMsg=1231'",2009-02-19,12:56:50,NULL,0,NULL
3759,3972,'tsoubrie','Afficher le contenu d''un message',"'IDForum=7,IDMsg=1231'",2009-02-24,15:51:01,NULL,0,NULL
20081,20347,'gachort','Afficher le contenu d''un message',"'IDForum=7,IDMsg=1231'",2009-04-07,14:21:05,NULL,0,NULL
